<h1>Titel.</h1>

We beginnen door een aantal libraries te importeren.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from mesa.batchrunner import BatchRunner

import trafficsim

In het model zit een willikeurig element, namelijk het willikeurig afremmen van verkeersdeelnemers. Om alsnog valide conclusies uit de simulatie te kunnen trekken voeren we deze meerdere keren uit, met behulp van de 'BatchRunner' die in Mesa ingebouwd zit. We beginnen met het 'standaard' 1-rijbaan model. We variëren de willikeurige kans om af te remmen, en de dichtheid van autos.


We werken met een rijbaanlengte van 50 'cellen'.

In [ ]:
LANE_LENGTH = 50
MAX_VELOCITY = 5

In [ ]:
variable_params = {"agents_n": np.arange(1, LANE_LENGTH, 2),
                   "p_brake": np.linspace(0, 1, 10, endpoint=False)}

In [ ]:
fixed_params = {"lane_length": LANE_LENGTH,
                "max_velocity": MAX_VELOCITY}

In [ ]:
batch_runner_1 = BatchRunner(
    trafficsim.OneLaneRoad,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=50,
    max_steps=50,
    model_reporters={
        "Data Collector": lambda m: m.data_collector  # Grab the data collector from every run.
    }
)

batch_runner_1.run_all()

In [ ]:
df_1 = batch_runner_1.get_model_vars_dataframe()
df_1

Bij elke keer dat het expiriment is uitgevoerd is een <i>Data Collector</i> opgeslagen. De informatie over de gemiddelde snelheid op elke tijdstap in dat expiriment kan daar uit gehaald worden, en gebruikt worden om een gemiddelde snelheid voor die uitvoeringen te bepalen. Hiervoor nemen we alle tijdstappen vanaf t = 5 mee, zodat alle autos de mogelijkheid hebben gehad de maximale snelheid te bereiken mits er niets in hun weg is.

In [ ]:
velocities = df_1["Data Collector"].apply(lambda d: d.get_model_vars_dataframe())

In [ ]:
df_1["Average velocity"] = velocities.apply(lambda s: s[MAX_VELOCITY:].mean())

In [ ]:
df_1.head(5)

Om de onderzoeksvraag op een valide manier te kunnen beantwoorden, is het handig de hoeveelheid agents in de simulatie om te zetten naar een dichtheid.

In [ ]:
df_1["Density"] = df_1["agents_n"] / LANE_LENGTH

Vervolgens kunnen we voor elke permutatie van de variabele parameters (de hoeveelheid agents in de simulatie, en de kans om elke tijdstap willikeurig af te remmen) het gemidddeld van alle geobserveerde gemiddelde snelheden over alle herhalingen van dezelfde instellingen.

In [ ]:
df_1_grouped = df_1.groupby(["Density", "p_brake"])["Average velocity"].mean()
df_1_grouped

Vervolgens kunnen we de verschillende waardes voor elke instelling van p_brake de gemiddelde snelheid per dichtheid laten zien.

In [ ]:
fig, ax = plt.subplots()

for p in variable_params["p_brake"]:
    ax.plot(df_1_grouped[(slice(None), p)], "x-", label=p)

ax.legend(
    title="P(willikeurige rem)",
    bbox_to_anchor=(1.5, 1)
)
ax.set_title("Gemiddelde snelheid per dichtheid op 1-baansweg.")
ax.set_xlabel("Dichtheid (autos per cel)")
ax.set_ylabel("Gemiddelde snelheid (cellen per tijdseenheid)")

<h2>2-Baans</h2>

Om een realistischer element aan de simulatie toe te voegen, en te kunnen kijken of dat van invloed is op de loop van filevorming kunnen we ook kijken naar wegen met 2 rijstroken. Om dit te onderzoeken kunnen we dezelfde stappen als eerder ondernemen. Omdat we op deze weg bij dezelfde lengte 2 keer zoveel cellen krijgen, passen we de hoeveelheden autos aan om te testen.

In [ ]:
variable_params_2 = variable_params
variable_params_2["agents_n"] = np.arange(1, LANE_LENGTH * 2, 4)

We voeren het expiriment wederom uit.

In [ ]:
batch_runner_2 = BatchRunner(
    trafficsim.TwoLaneRoad,
    variable_parameters=variable_params_2,
    fixed_parameters=fixed_params,
    iterations=50,
    max_steps=50,
    model_reporters={
        "Data Collector": lambda m: m.data_collector  # Grab the data collector from every run.
    }
)

batch_runner_2.run_all()

We halen hier de datacollectors uit, welke we vervolgens gebruiken om de gemiddelde snelheden per uitvoering van het expiriment te bepalen.

In [ ]:
df_2 = batch_runner_2.get_model_vars_dataframe()
velocities_2 = df_2["Data Collector"].apply(lambda d: d.get_model_vars_dataframe())
df_2["Average velocity"] = velocities_2.apply(lambda s: s[MAX_VELOCITY:].mean())

We rekenen de hoeveelheid agents om naar dichtheid.

In [ ]:
df_2["Density"] = df_2["agents_n"] / (LANE_LENGTH * 2)

En rekenen per permutatie van de instellingen de gemiddeldes uit, en beelden deze af in een grafiek.

In [ ]:
df_2_grouped = df_2.groupby(["Density", "p_brake"]).agg({"Average velocity": "mean"})
df_2_grouped

In [ ]:
fig, ax = plt.subplots()

for p in variable_params["p_brake"]:
    ax.plot(df_2_grouped[(slice(None), p)], "x-", label=p)

ax.legend(
    title="P(willikeurige rem)",
    bbox_to_anchor=(1.5, 1)
)
ax.set_title("Gemiddelde snelheid per dichtheid op 2-baansweg.")
ax.set_xlabel("Dichtheid (autos per cel)")
ax.set_ylabel("Gemiddelde snelheid (cellen per tijdseenheid)")